In [29]:
import tensorflow as tf

import io
import os
import unicodedata
import re
import mojimoji
from spacy.lang.ja import Japanese
from sklearn.model_selection import train_test_split

### Import dataset

In [2]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[w for w in l.split('\t')]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [3]:
# move current directory to get access to dataset directory
benchmark_directory = os.getcwd()
os.chdir(os.path.join(benchmark_directory, './..'))

In [4]:
# start with 30000 for local training
en, jp = create_dataset('./datasets/jesc-corpus.txt', 30000)

### Preprocess Text

In [7]:
# Tokenize Japanese text (since Japanese doesn't naturally put spaces between words)

# reference: https://github.com/WorksApplications/SudachiPy
# Load SudachiPy with split mode B: "国家公務員" => ['国家', '公務員']
# default is split mode A: "国家公務員" => ['国家公務員']
# NOTE: this may be worth adjusting in future training
jcfg = {"split_mode": "B"}
j_tokenizer = Japanese(meta={"tokenizer": {"config": jcfg}})

def tokenize_jp_sentence(text):
    return " ".join([i.text for i in j_tokenizer(text)])

In [8]:
print(jp[0])
print(tokenize_jp_sentence(jp[0]))

あなたは戻ったのね ハロルド?
あなた は 戻っ た の ね ハロルド ?


In [9]:
# convert any half-width katakana to normal-width katakana using mojimoji library
def norm_kt(text):
    return mojimoji.han_to_zen(text)

In [10]:
print("ﾆｭｰﾗﾙﾈｯﾄﾜｰｸ: " + norm_kt("ﾆｭｰﾗﾙﾈｯﾄﾜｰｸ"))

ﾆｭｰﾗﾙﾈｯﾄﾜｰｸ: ニューラルネットワーク


In [11]:
# convert unicode to ascii
def jp_unicode_to_ascii(text):
    return ''.join(ascii_text for ascii_text in unicodedata.normalize('NFKD', text))

# remove any accented characters for English-language text
def en_unicode_to_ascii(text):
    return ''.join(ascii_text for ascii_text in unicodedata.normalize('NFKD', text)
                   .encode('ascii', 'ignore').decode('utf-8', 'ignore'))

In [12]:
print(en_unicode_to_ascii("It's in my résumé."))
print(jp_unicode_to_ascii("それは履歴書にあります。"))

It's in my resume.
それは履歴書にあります。


In [18]:
# keep only Kanji, Hiragana, Katakana, numerals, and common punctuation: ("。", "、", "?", "!", "！"))
def jp_preprocessing_and_spacing(text):
    text = re.sub(r"([。、?!！])", r" \1", text)
    pattern = r"[^\u3041-\u309F\u30A1-\u30FF\uFF66-\uFF9F\u4E00-\u9FD0\u309B\u3099\uFF9E\u309C\u309A\uFF9F?!！\s、。.,0-9]+"
    text = re.sub(pattern, '', text).rstrip().strip()

    # add spaces between words and punctuation
    text = re.sub(r'[" "]+', " ", text)
    # remove interpunct (黒丸)
    text = text.replace("・" , "")

    text = text.lower()

    return text

# remove special characters and place spaces between words and punctuation
def en_preprocessing_and_spacing(text):
    text = en_unicode_to_ascii(text.lower().strip())

    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    text = re.sub(r"([?.!,])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    text = re.sub(r"[^a-zA-Z?.!,]+", " ", text)
    text = text.strip()

    return text

In [19]:
print(en_preprocessing_and_spacing('Hello, email@world!'))
print(jp_preprocessing_and_spacing('こんにちは、エメール＠世界！'))

hello , email world !
こんにちは 、エメール世界 ！


### Normalize Text

In [20]:
# utilize preprocessing functions and mark start and end of sentences
def normalize_text(japanese_text, english_text):
    
    inputs = []
    targets = []
    
    for jp_text, en_text in zip(japanese_text, english_text):

        # normalize Japanese
        jp_text = jp_unicode_to_ascii(jp_text)
        jp_text = jp_preprocessing_and_spacing(jp_text)
        jp_text = tokenize_jp_sentence(jp_text)
        jp_text = norm_kt(jp_text)

        jp_text = "<start> " + jp_text + " <end>"
        
        inputs.append(jp_text)
        
        # normalize English
        en_text = en_unicode_to_ascii(en_text)
        en_text = en_preprocessing_and_spacing(en_text)

        en_text = "<start> " + en_text + " <end>"
        targets.append(en_text)

    return inputs, targets

In [21]:
inputs, targets = normalize_text(jp, en)

In [23]:
for i in range(5):
    print(inputs[i])
    print(targets[i])

<start> あなた　は　戻っ　た　の　ね　ハロルド　？ <end>
<start> you are back , aren t you , harold ? <end>
<start> 俺　の　相手　は　シャーク　だ　。 <end>
<start> my opponent is shark . <end>
<start> 引き換え　だ　ある　事　と　ある　物　の <end>
<start> this is one thing in exchange for another . <end>
<start> もう　いい　よ　ごちそう　さま　ううん <end>
<start> yeah , i m fine . <end>
<start> もう　会社　に　は　来　ない　で　くれ　電話　も　する　な <end>
<start> don t come to the office anymore . don t call me either . <end>


### Tokenize text

In [24]:
def tokenize(lang):
    # vectorize a text corpus
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters=' ')

    # updates internal vocabulary based on a corpus
    lang_tokenizer.fit_on_texts(lang)

    # Transforms each text in texts to a sequence of integers.
    tensor = lang_tokenizer.texts_to_sequences(lang)

    # Pads sequences to the same length.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                          padding='post')
    return tensor, lang_tokenizer

In [25]:
print(jp[9])
print(en[9])
tokenize([inputs[9], targets[9]])

カンパニーの元社員が
it seems a former employee...


(array([[2, 4, 3, 0, 0, 0, 0, 0, 0, 0],
        [2, 5, 6, 7, 8, 9, 1, 1, 1, 3]], dtype=int32),
 <keras_preprocessing.text.Tokenizer at 0x7faba2ee41d0>)

In [27]:
input_tensor, input_lang_tokenizer = tokenize(jp)
target_tensor, target_lang_tokenizer = tokenize(en)

### Create input and target datasets

In [55]:
# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [66]:
# Creating train-test-validation splits
# Reference: https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is 75% of the entire data set
input_tensor_train, input_tensor_test, \
    target_tensor_train, target_tensor_test = train_test_split(input_tensor, target_tensor,
                                                               test_size=1 - train_ratio,
                                                               random_state=1)

# test is 10% of the initial data set
# validation is 15% of the initial data set
input_tensor_val, input_tensor_test, \
    target_tensor_val, target_tensor_test = train_test_split(input_tensor_test, target_tensor_test,
                                                             test_size=test_ratio/(test_ratio + validation_ratio)) 

# Show length
print(len(input_tensor_train), len(target_tensor_train), 
      len(input_tensor_val), len(target_tensor_val), 
      len(input_tensor_test), len(target_tensor_test))

22500 22500 4500 4500 3000 3000


In [57]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [58]:
print ("Input Language; index to word mapping")
convert(input_lang_tokenizer, input_tensor_train[1])
print ()
print ("Target Language; index to word mapping")
convert(target_lang_tokenizer, target_tensor_train[1])

Input Language; index to word mapping
7401 ----> 僕はただ...
37 ----> .

Target Language; index to word mapping
20 ----> it's
589 ----> just...


### Create a tf.dataset

In [62]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(input_lang_tokenizer.word_index)+1
vocab_tar_size = len(target_lang_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [63]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 10]), TensorShape([64, 44]))

### Create Encoder and Decoder Models

In [64]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [65]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 10, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [67]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [69]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 10, 1)


In [70]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

In [71]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 26808)


In [72]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)